In [ ]:
#|default_exp consultas

In [ ]:
#|export
import re
from typing import Union, Iterable

import requests
import pandas as pd

from fastcore.basics import store_attr
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import Path
import typer
#from fastcore.script import call_parse, Param

BASEURL = "http://webservicesintranet{}.anatel.gov.br/receita/rest/"
AMBIENTE = {'ds', 'hm', 'su', 'pd'}
TIPO = {'cpf': "obterPessoaFisica", 'cnpj': "obterPessoaJuridica"}
CPF_SIMPLES = ('cpf', 'nome', 'nomeMae', 'dataNascimento', 'estrangeiro',
               'tituloEleitor', 'dataAtualizacao', 'dataRegistroAnatel', 
               'resultado', 'unidadeAdministrativaCodigo', 'anoObito', 'erro')
SEXO =  {1: 'Masculino', 2: 'Feminino', 9 : 'Não encontrado'}
TAMANHO = {11: 'cpf', 14: 'cnpj'}

## Validador de CPF | CPNJ 

In [ ]:
#|hide
def valida_cpf(cpf: str):
    """Valida o cpf"""
    assert len(cpf) == TAMANHO['cpf'], f"CPF inválido, digite um CPF válido com {TAMANHO['cpf']} dígitos"  

def valida_cnpj(cnpj: str):
    """Valida o cnpj"""
    assert len(cnpj) == TAMANHO['cnpj'], f"CNPJ inválido, digite um CNPJ válido com {TAMANHO['cnpj']} dígitos"


## Classe Principal para Requisição

In [ ]:
#|export 
class Requisicao:
    def __init__(self, 
                 cpf_usuario: Union[str, int], # CPF do usuário requisitante
                 ambiente: str = 'ds', # Ambiente onde realizar a requisição: ds: desenvolvimento, hm: homologação, su: sustentação, pd: produção
                 origem: str = None, # Texto com identificação da requisição: e.g. "Teste"
                 cache: int = 36, # Tempo de expiração do cache em meses (Produção)
    ):
        """Classe para fazer requisições ao Web Service Receita - WS da Anatel, este encapsula o Infoconv da Receita Federal"""
        store_attr()
        assert isinstance(self.origem, str), "origem não pode ficar vazio e deve ser uma string de até 30 caracteres"
        self.origem = self.origem[:30]
        assert (ambiente := self.ambiente.lower()) in AMBIENTE, f"Ambiente inválido, escolha uma das opções {AMBIENTE}"
        if ambiente == 'pd':
            self.ambiente = ''
        self.cpf_usuario = ''.join(re.findall("\d", self.cpf_usuario))

    def formatar_url(self,
                  identificador, # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    )-> str: # Url formatada para a requisição GET REST
        """Recebe o identificador CPF|CNPJ e formata retorna a url formatada da requisição"""
        identificador = ''.join(re.findall("\d", identificador))
        if (tipo := len(identificador)) not in TAMANHO:
            raise ValueError("Query inválida, CPF possui 11 caracteres e CNPJ possui 14 caracteres")
        tipo = TAMANHO[tipo] #TODO: Validar refinadamente o CPF / CNPJ
        req_TIPO = TIPO[tipo]
        suffix = ''
        if self.cache is not None:
            try:
                cache = int(self.cache)
                assert cache >= 0, "Tempo de expiração do cache inválido, escolha um número inteiro maior que zero"
            except ValueError as e:
                raise ValueError("Valor inválido de expira_cache, escolha um número inteiro maior que zero") from e
            req_TIPO += 'IgnoraCacheAntigo'
            suffix = f'&mesesExpiraCache={cache}'

        return f'{BASEURL.format(self.ambiente)}{req_TIPO}?{tipo}={identificador}&cpfUsuario={self.cpf_usuario}{suffix}&origem={self.origem}'

    @staticmethod
    def nivelar_json(json_dict: dict, # Dicionário no formato json retornado pela requisição
    )-> dict: #Dicionário nivelado para formato tabular
        """Recebe um dicionário json com diferentes níveis e retorna os registros em um único nível"""
        d = {}
        for k,v in json_dict.items():
            if isinstance(v, list): 
                v = '|'.join(v)
            if isinstance(v, dict):
                d.update({f'{k}.{sk}': sv for sk, sv in v.items()})
                continue
            d[k] = v
        return {k:v.lstrip().rstrip() if isinstance(v, str) else v for k, v in d.items()}

    @staticmethod
    def _get_Requisicao(url: str, # Url formatada da requisição 
    )-> dict:
        """Efetua a requisição na url and retorna a resposta json"""
        r = requests.get(url)
        if r.status_code == 200 and r.headers['content-type'] == 'application/json':
            return Requisicao.nivelar_json(r.json())
        return {}

    def consultar(self, 
                 identificador: Union[str, int], # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    )-> dict: # Dicionário com o resultado da requisição
        """Efetua a requisição do identificador e retorna um dicionário"""
        return Requisicao._get_Requisicao(self.formatar_url(identificador))
    
    def consultar_em_lote(self, 
                      queries: Iterable[Union[str, int]], # Lista com os identificadores: CPF ou CNPJ
    )->Iterable[dict]: # Lisa com os dicionários resultantes das requisições                        
        return L(queries).map(self.consultar)
        

## Exemplos

### CPF

In [ ]:
#|eval: false
from fastcore.test import ExceptionExpected, test_eq

In [ ]:
#|eval: false
cpf_usuario = input('Digite o CPF do solicitante: ')
identificador = input('Digite o CPF a ser consultado')
origem = "Teste REST API"
ambiente = 'ds'

In [ ]:
#|eval: false
req = Requisicao(cpf_usuario, ambiente, origem)

In [ ]:
#|eval: false
r = req.consultar(identificador)

In [ ]:
#|eval: false
del r['cpf'] # Não expõe o CPF no código
r

{'nome': 'Tow Dh Vrmadfgixfxaxqupjkfp',
 'situacaoCadastral.codigo': 8,
 'situacaoCadastral.valor': 'Nula',
 'paisResidencia.residenteExterior': True,
 'paisResidencia.codigoPais': 0,
 'nomeMae': 'Bvgoxicrewhewdrt',
 'dataNascimento': '1917-06-28',
 'sexo.codigo': 2,
 'sexo.valor': 'Feminino',
 'ocupacao.naturezaOcupacaoCodigo': 5,
 'ocupacao.ocupacaoPrincipalCodigo': 966,
 'ocupacao.exercicioOcupacao': 2736,
 'endereco.logradouro': 'Jgp Vjfrvmytkvnnqskehmrg',
 'endereco.numero': '098',
 'endereco.cep': '93700734',
 'endereco.bairro': 'Pahcjbobur',
 'endereco.codigoMunicipio': 3519600,
 'endereco.nomeMunicipio': 'Hobkpc',
 'endereco.uf': 'RO',
 'telefone.ddd': '0025',
 'telefone.numero': '97620714',
 'unidadeAdministrativaCodigo': '7431014',
 'anoObito': 0,
 'estrangeiro': False,
 'tituloEleitor': '0000000000000',
 'dataAtualizacao': '1964-12-23',
 'dataRegistroAnatel': '2022-08-16',
 'resultado': 'CPF encontrado',
 'erro': ''}

In [ ]:
#|eval: false
cpfs = req.consultar_em_lote([cpf_usuario, identificador])

In [ ]:
#|eval: false
cpfs = cpfs.map(lambda x: {k:v for k,v in x.items() if k != 'cpf'}) # Não expõe os CPFs


In [ ]:
#|eval: false
cpfs

(#2) [{'nome': 'Tuwpoxtzyzzruazdjdsqmaewylowhy', 'situacaoCadastral.codigo': 2, 'situacaoCadastral.valor': 'Suspensa', 'paisResidencia.residenteExterior': True, 'paisResidencia.codigoPais': 0, 'nomeMae': 'Sjflu Ifgemqkhdjvpgcjewylowhy', 'dataNascimento': '1937-10-25', 'sexo.codigo': 1, 'sexo.valor': 'Masculino', 'ocupacao.naturezaOcupacaoCodigo': 36, 'ocupacao.ocupacaoPrincipalCodigo': 320, 'ocupacao.ocupacaoPrincipalDescricao': 'Técnico em biologia', 'ocupacao.exercicioOcupacao': 9236, 'endereco.logradouro': 'Viy Emmkgotopwc Yyoamytp', 'endereco.numero': '598', 'endereco.cep': '37377234', 'endereco.bairro': 'Qqukpfkgfti', 'endereco.codigoMunicipio': 2931350, 'endereco.nomeMunicipio': 'Lkgwrl', 'endereco.uf': 'SP', 'telefone.ddd': '0025', 'telefone.numero': '31297214', 'unidadeAdministrativaCodigo': '1008514', 'anoObito': 0, 'estrangeiro': False, 'tituloEleitor': '0000000000000', 'dataAtualizacao': '1962-02-17', 'dataRegistroAnatel': '2022-08-16', 'resultado': 'CPF encontrado', 'erro':

In [ ]:
#|eval: false
pd.DataFrame(cpfs)

,nome,situacaoCadastral.codigo,situacaoCadastral.valor,paisResidencia.residenteExterior,paisResidencia.codigoPais,nomeMae,dataNascimento,sexo.codigo,sexo.valor,ocupacao.naturezaOcupacaoCodigo,...,telefone.ddd,telefone.numero,unidadeAdministrativaCodigo,anoObito,estrangeiro,tituloEleitor,dataAtualizacao,dataRegistroAnatel,resultado,erro
0,Tuwpoxtzyzzruazdjdsqmaewylowhy,2,Suspensa,True,0,Sjflu Ifgemqkhdjvpgcjewylowhy,1937-10-25,1,Masculino,36,...,0025,31297214,1008514,0,False,0000000000000,1962-02-17,2022-08-16,CPF encontrado,
1,Tow Dh Vrmadfgixfxaxqupjkfp,8,Nula,True,0,Bvgoxicrewhewdrt,1917-06-28,2,Feminino,5,...,0025,97620714,7431014,0,False,0000000000000,1964-12-23,2022-08-16,CPF encontrado,


### CNPJ


CNPJ somente números

In [ ]:
#|eval: false
req.consultar('00280273000137')

{'cnpj': '00280273000137',
 'matriz': True,
 'nomeEmpresarial': 'Samsung Eletronica da Amazonia Ltda',
 'nomeFantasia': 'Samsung da Amazonia',
 'situacaoCadastral.codigo': '02',
 'situacaoCadastral.valor': 'Ativa',
 'situacaoCadastral.dataAlteracao': '2005-11-03T00:00:00',
 'naturezaJuridica.codigo': '2062',
 'naturezaJuridica.descricao': 'SOCIEDADE EMPRESARIA LIMITADA',
 'dataAbertura': '1994-10-31',
 'cnaePrincipal': '2640000',
 'cnaeSecundario': '1922599|2621300|2622100|2632900|2670102|3313999|4618402|4618499|4645101|4649401|4649402|4649499|4651601|4651602|4652400|5211799|6202300|6203100|7490199|7739002|7739099|8211300|9511800|9512600|9521500|9529199|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|00

CNPJ com separadores

In [ ]:
#|eval: false
req.consultar('02.030.715/0001-12')

{'cnpj': '02030715000112',
 'matriz': True,
 'nomeEmpresarial': 'H Qyccquxlammrnsnssjydjvwdhlqqhjeaee',
 'nomeFantasia': 'Fwcfey',
 'situacaoCadastral.codigo': '03',
 'situacaoCadastral.valor': 'Suspensa',
 'situacaoCadastral.dataAlteracao': '1938-07-30T00:00:00',
 'naturezaJuridica.codigo': '3525',
 'dataAbertura': '1906-03-01',
 'cnaePrincipal': '9755621',
 'endereco.logradouro': 'Vccqrn H Dctuseqytj8jgkmthhqu[m)oxrvu',
 'endereco.numero': '31',
 'endereco.complemento': "Luug'mettlqhh'",
 'endereco.cep': '81312361',
 'endereco.bairro': 'Ixjxeuy',
 'endereco.codigoMunicipio': 4301073,
 'endereco.nomeMunicipio': 'Ezfakyin',
 'endereco.uf': 'ES',
 'telefone1.ddd': '82',
 'telefone1.numero': '34464555',
 'telefone2.ddd': '82',
 'telefone2.numero': '34463374',
 'email': "rohyx'rgqowo<lxx,bd",
 'responsavel.cpf': '50368609502',
 'responsavel.nome': 'Phj Bg Lzdnugmeinpqdrv',
 'capitalSocial': '1',
 'porte.codigo': 3,
 'porte.valor': 'Empresa de pequeno porte',
 'opcaoSimples.opcaoSimples': 

### API para consulta em lote

In [ ]:
#|export
def salvar_requisicao(results: Iterable, # Lista com o retorno das requisições
                 saida: str, # Nome do Arquivo de Saída
)->None:
    """Salva a lista de requisições `results` no arquivo `saida`"""
    df = pd.DataFrame(results)
    try:
        saida = Path(saida)
    except TypeError:
        saida = Path.cwd() / 'resultados.csv'

    match suffix := saida.suffix:
        case '.csv' | '.txt':
            df.to_csv(saida, index=False)
        case '.xlsx':
            df.to_excel(saida, index=False, engine='openpyxl')
        case '.json':
            df.to_json(saida)
        case '.md':
            df.to_markdown(saida, index=False)
        case '.html':
            df.to_html(saida, index=False)
        case _:
            df.to_csv(saida, index=False)

def requisitar_em_lote(filename: str, # Arquivo texto de entrada: 1 CPF | CNPJ por linha
                        cpf_usuario: str, # CPF do usuário requisitante
                        ambiente: str = 'ds', # Ambiente onde realizar a requisição: ds | hm | su | pd 
                        origem: str = None, # Texto com identificação da requisição: e.g. 'Teste'
                        cache: int = 36, # Tempo de expiração do cache em meses 
                        saida: str = None, # Arquivo de saída da requisição
)->None:
    """Lê o arquivo `filename` com um CPF | CPNJ por linha. Faz a requisição no `ambiente` do receita-ws e salva os resultados em `saida`"""
    conteudo = Path(filename).readlines()
    req = Requisicao(cpf_usuario, ambiente, origem, cache)
    resultado = req.consultar_em_lote(conteudo)
    salvar_requisicao(resultado, saida)

## Testes Unitários

Sem string de identificação origem

In [ ]:
#|eval: false
with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
    Requisicao(cpf_usuario=cpf_usuario)

origem não é uma string

In [ ]:
#|eval: false
with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
    Requisicao(cpf_usuario=cpf_usuario, origem = 50)

ambiente inválido

In [ ]:
#|eval: false
with ExceptionExpected(ex=AssertionError, regex="Ambiente inválido"):
    Requisicao(cpf_usuario=cpf_usuario, origem='Teste', ambiente='ts')

Teste de formatação da url corretamente de acordo com o ambiente

In [ ]:
#|eval: false
origem = 'Teste'
for ambiente in AMBIENTE:
    req = Requisicao(cpf_usuario=cpf_usuario, origem=origem, ambiente=ambiente)
    url = req.formatar_url(identificador)
    if ambiente == 'pd': 
        ambiente = ''
    test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/obterPessoaFisicaIgnoraCacheAntigo?cpf={identificador}&cpfUsuario={cpf_usuario}&mesesExpiraCache=36&origem={origem}')

Teste de ambientes x tipos de requisição

In [ ]:
#|eval: false
from itertools import product
origem = 'Teste'
cache = 3
for (ambiente, tipo) in product(AMBIENTE, TIPO):
    req = Requisicao(cpf_usuario=cpf_usuario,
                  ambiente=ambiente, 
                  origem=origem, 
                  cache=cache)
    id_ = identificador if tipo == 'cpf' else '02030715000112'
    url = req.formatar_url(id_)
    if ambiente == 'pd': 
        ambiente = ''
    test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/{TIPO[tipo]}IgnoraCacheAntigo?{tipo}={id_}&cpfUsuario={cpf_usuario}&mesesExpiraCache={cache}&origem={origem}')

In [ ]:
#|eval: false
origem = 'Teste'
cache = None
for (ambiente, tipo) in product(AMBIENTE, TIPO):
    req = Requisicao(cpf_usuario=cpf_usuario,
                  ambiente=ambiente, 
                  origem=origem, 
                  cache=cache)
    id_ = identificador if tipo == 'cpf' else '02030715000112'
    url = req.formatar_url(id_)
    if ambiente == 'pd': 
        ambiente = ''
    test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/{TIPO[tipo]}?{tipo}={id_}&cpfUsuario={cpf_usuario}&origem={origem}')

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()